In [1]:
import pandas as pd

In [2]:
review_data = pd.read_csv('Data/sentiment-data/reviews.txt', sep='\t',header=None)

In [3]:
label_data = pd.read_csv('Data/sentiment-data/labels.txt',header=None)

In [4]:
review_data.columns = ['text']

In [5]:
label_data.columns = ['sentiment']

In [6]:
comp_review_data = review_data.join(label_data)

### Converting data into lower case

In [7]:
comp_review_data['text'] = comp_review_data.text.str.lower()

### Removing punchutations

In [8]:
import string

In [9]:
tab = ''.maketrans(string.punctuation, ' '*len(string.punctuation))

In [10]:
comp_review_data['text'] = comp_review_data.text.map(lambda s:s.translate(tab))

### Minimize words using lemma or stemming

### Removing single character words

### Use tokenizers

In [11]:
from keras.preprocessing.text import Tokenizer

C:\Users\awant\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
tokenizer = Tokenizer()

In [13]:
tokenizer.fit_on_texts(comp_review_data.text)

In [30]:
vocab = len(tokenizer.word_index)

In [15]:
review_tokens = tokenizer.texts_to_sequences(comp_review_data.text)

In [16]:
len(review_tokens[0])

140

In [17]:
len(comp_review_data.text[0].split())

140

In [18]:
from keras.preprocessing.sequence import pad_sequences

In [19]:
review_tokens_pad = pad_sequences(review_tokens, maxlen=1500)

In [20]:
len(review_tokens_pad[0])

1500

In [21]:
review_tokens_pad

array([[    0,     0,     0, ...,     8,   215,    23],
       [    0,     0,     0, ...,    29,   108,  3324],
       [    0,     0,     0, ...,     5,   336,   400],
       ...,
       [    0,     0,     0, ..., 45313,  1175,   781],
       [    0,     0,     0, ...,    81,    95,    38],
       [    0,     0,     0, ...,    11,     6,  1321]])

### Dealing with target data

In [22]:
from keras.utils import to_categorical

In [23]:
comp_review_data.sentiment = comp_review_data.sentiment.map({'positive':1,'negative':0})

In [24]:
target = to_categorical(comp_review_data.sentiment)

### Model Defination

In [54]:
from keras.layers import Embedding,Dense,Conv1D,Flatten

In [26]:
from keras.models import Sequential

In [41]:
#help(Embedding)

In [61]:
model = Sequential()
model.add(Embedding(vocab,128, input_length=1500))
model.add(Conv1D(64,5,activation='relu'))
model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1500, 128)         9481216   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1496, 64)          41024     
_________________________________________________________________
flatten_2 (Flatten)          (None, 95744)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 191490    
Total params: 9,713,730
Trainable params: 9,713,730
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
trainX,testX,trainY,testY = train_test_split(review_tokens_pad,target)

In [67]:
model.fit(review_tokens_pad,target, batch_size=128, epochs=5, validation_split=.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
19968/20000 [============================>.] - ETA: 0s - loss: 0.4127 - acc: 0.7855

InvalidArgumentError: indices[6,1420] = 74072 is not in [0, 74072)
	 [[Node: embedding_5/GatherV2 = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_5/embeddings/read, embedding_5/Cast, embedding_5/GatherV2/axis)]]

Caused by op 'embedding_5/GatherV2', defined at:
  File "C:\Users\awant\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\awant\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\awant\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\awant\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\awant\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\awant\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py"

In [53]:
trainX.shape

(18750, 1500)